# A*-search using `aima-python`

Algorithm implementations taken from [here](https://github.com/aimacode/aima-python/blob/master/search-4e.ipynb)

* *State* is defined by gifts in bags

* *Goal states* are defined by filled bags satisfying problem conditions

* *Actions* : put a gift in a bag with a minimal weight

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.random.seed(2016)

In [3]:
from search import Problem, astar_search, uniform_cost_search, state_sequence, action_sequence
import sys
sys.path.append('../common')
from utils import weight2 as weight_fn

In [157]:
n_bags = 5
max_weight = 50


n_horses = 1000
n_balls = 1100
n_bikes = 500
n_trains = 1000
n_coals = 166
n_books = 1200
n_dolls = 1000
n_blocks = 1000
n_gloves = 200

available_gifts = {
    "horse": ['horse_%i' % i for i in range(n_horses)],
    "ball": ['ball_%i' % i for i in range(n_balls)],
    "bike": ['bike_%i' % i for i in range(n_bikes)],
    "train": ['train_%i' % i for i in range(n_trains)],
    "coal": ['coal_%i' % i for i in range(n_coals)],
    "book": ['book_%i' % i for i in range(n_books)],
    "doll": ['doll_%i' % i for i in range(n_dolls)],
    "blocks": ['blocks_%i' % i for i in range(n_blocks)],
    "gloves": ['gloves_%i' % i for i in range(n_gloves)],
}

type_cost = {
    "horse": 0.5,
    "ball": 0.5,
    "bike": 2,
    "train": 0.75,
    "coal": 2,
    "book": 3,
    "doll": 2,
    "blocks": 2,
    "gloves": 3    
}

gift_types = available_gifts.keys()
n_types = len(gift_types)

In [158]:
len(gift_types), gift_types

(9,
 ['bike',
  'horse',
  'ball',
  'blocks',
  'coal',
  'gloves',
  'train',
  'doll',
  'book'])

State is tuple (bags) of tuples (gifts)

In [159]:
def bag_weight(bag, count=1, count2=50):
    weight = []
    for c in range(count):
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += weight_fn(gift_type, count2)
        weight.append(w)            
    return np.mean(weight)

In [160]:
[len(available_gifts[k]) for k in available_gifts]

[500, 1000, 1100, 1000, 166, 200, 1000, 1000, 1200]

In [161]:
class SantasBagsProblem(Problem):
            
    def actions(self, state):
        """Return a list of actions executable in this state."""                        
        # find a bag with a minimal weight                
        if np.sum([len(self.available_gifts[k]) for k in self.available_gifts]) < 1:
            return []
        
        min_weight_bag_index = 0
        min_weight = self.max_weight
        for i, bag in enumerate(state):
            w = self.bag_weight_fn(bag, count2=1)
            if min_weight > w:
                min_weight_bag_index = i
                min_weight = w
        
        actions = [(min_weight_bag_index, gift_type) for gift_type in self.gift_types]
#         print("actions: ", actions)
        return actions
    
    def result(self, state, action):
        """The state that results from executing this action in this state."""
        bag_id, gift_type  = action
        if len(self.available_gifts[gift_type]) < 1:
#             print("No more gifts of type : ", gift_type)
            return state
        
        print("-- result : input state: ", state, "action: ", action)
        bag_weight = self.bag_weight_fn(state[bag_id], count2=1)
        gift_weight = self.weight_fn(gift_type, 1)
        
        if bag_weight + gift_weight > self.max_weight:
            return state
                
        new_state = list(state)
        gift = self.available_gifts[gift_type].pop()
        bag = list(new_state[bag_id])
        bag.append(gift)
        new_state[bag_id] = tuple(bag)
        
        print("-- result : output state: ", new_state)
        return tuple(new_state)

    def is_goal(self, state):
        """True if the state is a goal."""        
        for bag in state:
            if len(bag) < 3:
                return False
        
        mean_score = self._validation(state)
        if mean_score > self.goal_score:
            print("Mean score : ", mean_score, " / ", self.goal_score)
        return mean_score > self.goal_score

    def step_cost(self, state, action, result=None):
        """The cost of taking this action from this state."""
        if self.type_cost is not None:
            bag_id, gift_type  = action
            if gift_type in self.type_cost:
                return self.type_cost[gift_type]  # Override this if actions have different costs
            return 1.0
        return 1.0
                
    def _validation(self, state, count=10):
        scores = np.zeros(count)
        for c in range(count):
            score = 0
            for bag in state:
                total_weight_ = self.bag_weight_fn(bag, count2=1)
                if total_weight_ < self.max_weight:
                    score += total_weight_
            scores[c] = score
        return np.mean(scores)

In [162]:
alpha = 0.75
goal_score = n_bags*max_weight*alpha
print("Goal score: ", goal_score)

('Goal score: ', 187.5)


In [163]:
from copy import deepcopy

In [164]:
initial_state=tuple([()]*n_bags)
p = SantasBagsProblem(initial=initial_state,
                      gift_types=list(gift_types)[:n_types], 
                      available_gifts=deepcopy(available_gifts),
                      max_weight=max_weight,    
                      type_cost=type_cost,
                      weight_fn=weight_fn,
                      bag_weight_fn=bag_weight,
                      goal_score=goal_score)

Define heuristic function :


In [165]:
gift_mean_weight = {}

for gift_type in gift_types:
    gift_mean_weight[gift_type] = weight_fn(gift_type, 100)    

In [166]:
def h1(state):
    h = 0
    for bag in state:
        w = bag_weight(bag, count2=10)
        h += max(max_weight*alpha/n_bags - w, 0.0)
    return h

def h2(state):     
    h = 0
    for bag in state:
        h += bag_weight(bag, count2=10)
    h = max(goal_score - h, 0.0) / n_bags
    return h

def h12(state):     
    h1 = 0
    h2 = 0
    for bag in state:
        h1 += bag_weight(bag, count2=10)
        w = bag_weight(bag)
        h2 += max(max_weight*alpha/n_bags - w, 0.0)
    h1 = max(goal_score - h1, 0.0) / n_bags    
    return max(h1, h2)


def h3(state):
    h = 0
    for bag in state:
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += gift_mean_weight[gift_type]
        h += max(max_weight/n_bags - w, 0.0)
    return h

def h4(state):     
    h = 0
    for bag in state:
        w = 0
        for gift in bag:
            gift_type = gift.split('_')[0]
            w += gift_mean_weight[gift_type]
        h += w
    h = max(max_weight*n_bags - h, 0.0) / n_bags
    return h


def h5(state):
    h = 0
    for bag in state:
        h += max_weight / 3 if len(bag) < 3 else 0
    return h


def h6(state):
    h = 20
    for bag in state:
        for gift in bag:
            gift_type = gift.split('_')[0]
            if gift_type == 'doll' or gift_type == 'horse':
                h -= 0.5
            elif gift_type == 'ball' or gift_type == 'book':
                h -= 2
    return h
                


def final_heuristic_fn(state):    
    return np.max(np.array([h12(state), h3(state), h4(state), h5(state), h6(state)]))
#     return np.max(np.array([h12(state), h3(state), h4(state), h5(state)]))

In [167]:
h12(initial_state), h3(initial_state), h4(initial_state), h5(initial_state), h6(initial_state)

(37.5, 50, 50, 80, 20)

In [168]:
from time import time

In [169]:
tic = time()
result = astar_search(p, final_heuristic_fn)
print(result)
print("Elapsed: ", time() - tic)

None
('Elapsed: ', 41.58973693847656)


In [170]:
result.state

AttributeError: 'NoneType' object has no attribute 'state'

In [103]:
h12(result.state), h3(result.state), h4(result.state), h5(result.state)

(0.0, 0.0, 2.812144967034294, 0)

In [118]:
p._validation(result.state) / n_bags

25.547646737114491

In [105]:
[len(available_gifts[k]) for k in available_gifts]

[500, 1000, 1100, 1000, 166, 200, 1000, 1000, 1200]

In [50]:
def update_available_gifts(ag, state):
    for bag in state:
        for gift in bag:
            gift_type = gift.split('_')[0] 
            ag[gift_type].remove(gift)
    assert np.sum([len(ag[k]) for k in ag]) > 0, "Available gifts problem : {}".format(ag)

In [51]:
# type_cost = {
#     "horse": 1.9,
#     "ball": 1.7,
#     "bike": 1.4,
#     "train": 1.2,
#     "coal": 0.8,
#     "book": 1.9,
#     "doll": 1.9,
#     "blocks": 0.5,
#     "gloves": 1.9    
# }

In [72]:
type_cost = {
#     "horse": 1.3,
#     "ball": 1.2,
#     "bike": 1.1,
#     "train": 1.2,
    "coal": 0.9,
#     "book": 1.0,
#     "doll": 1.0,
    "blocks": 0.9,
#     "gloves": 1.2    
}

In [73]:
# type_cost = {"horse": 0.9, "train": 0.9, "bike": 1.9, "book": 1.9, "gloves": 1.9, "ball": 1.9}

In [78]:
total_n_bags = 1000
n_bags = 5

alpha = 0.75
goal_score = n_bags*max_weight*alpha
print("Goal score: ", goal_score)

total_state=[]
ag=deepcopy(available_gifts)
counter = 0

('Goal score: ', 140.0)


In [79]:
# n_threads = 4 
# from multiprocessing import Process, Lock, Pool

# def astar_parallel(l, thread_id):
    
#     l.acquire()
#     _counter = counter
#     l.release()
    
#     while n_bags * _counter < total_n_bags:
#         state=[()]*n_bags
#         print(thread_id, "| Filled bags : ", n_bags * _counter, "/", total_n_bags)
#         p = SantasBagsProblem(initial=tuple(state),
#                               gift_types=gift_types, 
#                               available_gifts=deepcopy(_ag),
#                               max_weight=max_weight,    
#                               type_cost=type_cost,
#                               weight_fn=weight_fn,
#                               bag_weight_fn=bag_weight,
#                               goal_score=goal_score)
#         tic = time()
#         result = astar_search(p, final_heuristic_fn)
#         l.acquire()
#         if result is not None:
#             print(thread_id, " : Got a result")
#             total_state += result.state
#             counter += 1
#             _counter = counter
#             update_available_gifts(ag, result.state)
#         else:
#             type_cost = {"horse": 0.9, "train": 0.9, "gloves": 1.9, "ball": 1.9 }
#         l.release()
#         print("- Elapsed: ", time() - tic)
    


# lock = Lock()
# for thread_id in range(n_threads):
#     Process(target=astar_parallel, args=(lock, thread_id)).start()

In [106]:
def score(state, count=10):
    scores = np.zeros(count)
    for c in range(count):
        score = 0
        for bag in state:
            total_weight_ = bag_weight(bag, count2=1)
            if total_weight_ < max_weight:
                score += total_weight_
        scores[c] = score
    return np.mean(scores)

In [81]:
while n_bags * counter < total_n_bags:
    
    state=[()]*n_bags
    print("Filled bags : ", n_bags * counter, "/", total_n_bags)
    p = SantasBagsProblem(initial=tuple(state),
                          gift_types=gift_types, 
                          available_gifts=deepcopy(ag),
                          max_weight=max_weight,    
                          type_cost=type_cost,
                          weight_fn=weight_fn,
                          bag_weight_fn=bag_weight,
                          goal_score=goal_score)
    tic = time()
    result = astar_search(p, final_heuristic_fn)
    if result is not None:
        print("- Got a result")
        total_state += result.state
        counter += 1
        update_available_gifts(ag, result.state)
#     else:
#         type_cost = None
        
    if counter > 0 and (n_bags * counter % 10) == 0:
        s = score(total_state)
        print(">>> Current score: ", s, s * (total_n_bags) / (n_bags * counter) )
        
    print("- Elapsed: ", time() - tic)

('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.13291597366333)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.110548973083496)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.301083087921143)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.135909080505371)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.1509270668029785)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.1316070556640625)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.109704971313477)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.138660907745361)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.105802059173584)
('Filled bags : ', 0, '/', 1000)
('- Elapsed: ', 5.1512610912323)
('Filled bags : ', 0, '/', 1000)


KeyboardInterrupt: 

In [245]:
[len(ag[k]) for k in ag]

[1000, 1099, 1000, 996, 1000, 165, 499, 200, 1200]

In [202]:
len(total_state), total_state

(128,
 [('horse_999',
   'horse_996',
   'horse_995',
   'horse_993',
   'horse_990',
   'horse_988',
   'horse_987',
   'horse_985'),
  ('horse_998',
   'horse_997',
   'horse_994',
   'horse_992',
   'horse_991',
   'horse_989',
   'horse_986'),
  ('horse_984',
   'horse_982',
   'horse_980',
   'horse_978',
   'horse_975',
   'horse_974',
   'horse_971',
   'horse_969',
   'horse_968'),
  ('horse_983',
   'horse_981',
   'horse_979',
   'horse_977',
   'horse_976',
   'horse_973',
   'horse_972',
   'horse_970',
   'ball_1082'),
  ('horse_967',
   'horse_965',
   'horse_962',
   'horse_960',
   'horse_958',
   'horse_956',
   'horse_955',
   'horse_953'),
  ('horse_966',
   'horse_964',
   'horse_963',
   'horse_961',
   'horse_959',
   'horse_957',
   'horse_954',
   'horse_952'),
  ('horse_951',
   'horse_949',
   'horse_947',
   'horse_944',
   'horse_942',
   'horse_940',
   'horse_939',
   'horse_937'),
  ('horse_950',
   'horse_948',
   'horse_946',
   'horse_945',
   'horse_9

In [214]:
score(total_state), score(total_state) * (total_n_bags) / (n_bags * counter)

(4629.3953940358897, 73.596083061577872)

In [369]:
from datetime import datetime
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

In [370]:
def write_submission(state, filename):
    with open(filename, 'w') as w:
        w.write("Gifts\n")
        for bag in state:
            w.write(' '.join(bag) + '\n')
    
write_submission(total_state, submission_file)